### Test code for evaluating prompts on a dataset

In [1]:
# from src.prompt_utils import evaluate_prompts, MATH_SYSTEM_PROMPT

# results = evaluate_prompts(
#     dataset=dataset,
#     client=client,
#     system_prompt=MATH_SYSTEM_PROMPT,
#     output_path="evaluation_results.json",
#     max_workers=5
# )

# print("\nAccuracy Results:")
# for prompt_name, accuracy in results.items():
#     print(f"{prompt_name}: {accuracy:.2%}")

### Test code for creating word embeddings for Auto CoT

In [1]:
import rootutils
rootutils.setup_root(".",indicator=".project-root")
import sys
sys.path.append("..")
sys.path.append("../src")

from src.gpt_utils import initialize_openai_model
from src.data_utils import CoTDataModule

/Users/rissal.hedna/Desktop/Extras/Chain-of-Thought-Prompting/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = initialize_openai_model()

datamodule = CoTDataModule(dataset_name="openai/gsm8k")

subset = datamodule.data[:100]

Loaded dataset 'openai/gsm8k'.
Processed 'openai/gsm8k' dataset.


In [5]:
subset['answer'][0]

'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [6]:
from prompt_utils import getAutoCotQuestion

question = getAutoCotQuestion(subset['question'][0], subset)

Creating question vectors in parallel...


Creating embeddings: 100%|██████████| 100/100 [10:04<00:00,  6.05s/sample]


Clustering question vectors...
Finding representative points...


Finding representatives: 100%|██████████| 8/8 [00:00<00:00, 11646.80cluster/s]


In [9]:
print(question)

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?Here are some examples of how to answer the question: 
Example 1: Five friends eat at a fast-food chain and order the following: 5 pieces of hamburger that cost $3 each; 4 sets of French fries that cost $1.20; 5 cups of soda that cost $0.5 each; and 1 platter of spaghetti that cost $2.7. How much will each of them pay if they will split the bill equally?
Answer: The cost of 5 pieces of hamburger is $3 x 5 = $<<3*5=15>>15.
The cost of 4 sets of French fries is $1.20 x 4 = $<<1.20*4=4.80>>4.80.
The cost of 5 cups of soda is $0.5 x 5 = $<<0.5*5=2.50>>2.50.
So their total bill is $15 + $4.80 + $2.50 +$2.7 = $<<15+4.8+2.5+2.7=25>>25.
Hence, each of the five friends will contribute $25/5 = $<<25/5=5>>5.
#### 5

Now, let's think step by step. Example 2: Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 12